In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [1]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x


+ pip install konlpy
     |████████████████████████████████| 19.4 MB 1.5 MB/s 
     |████████████████████████████████| 86 kB 4.5 MB/s 
     |████████████████████████████████| 448 kB 26.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/l

# 자연어 학습과정 재현

### 카톡 대화 데이터

In [3]:
# 대화 업로드 
with open("/content/kakaotalk_text.txt") as f:
    talk = [str(line.strip().split()) for line in f]

with open("/content/KakaoTalk_text2.txt") as f:
    talk2 = [str(line.strip().split()) for line in f]

with open("/content/KakaoTalk_text3.txt") as f:
    talk3 = [str(line.strip().split()) for line in f]

talk = talk+talk2+talk3

In [3]:
talk

["['시끄러운', '팀브라더스', '님과', '카카오톡', '대화']",
 "['저장한', '날짜', ':', '2021-12-21', '17:11:09']",
 '[]',
 "['---------------', '2021년', '5월', '3일', '월요일', '---------------']",
 "['[김승하]', '[오전', '9:03]', '아거', '귀찌는', '없지않나요?']",
 "['[민영]', '[오전', '9:03]', '아......']",
 "['[민영]', '[오전', '9:03]', '그냥', '귀를', '하...을...까']",
 "['[김승하]', '[오전', '9:03]', '??']",
 "['[김승하]', '[오전', '9:03]', '귀를', '뚫어봐요', '민영님']",
 "['[서누기', '인나청]', '[오전', '9:03]', '그것도힘들져ㅠㅠ']",
 "['[서누기', '인나청]', '[오전', '9:04]', '어린이날은주말이니ㅠㅠㅠ']",
 "['[시끄러운나그네]', '[오전', '9:04]', '??']",
 "['[시끄러운나그네]', '[오전', '9:05]', '귀뚫을때']",
 "['[민영]', '[오전', '9:05]', '그를까...']",
 "['[민영]', '[오전', '9:05]', '요']",
 "['[시끄러운나그네]', '[오전', '9:05]', '뚫!!!']",
 "['[김승하]', '[오전', '9:05]', '?']",
 "['[시끄러운나그네]', '[오전', '9:05]', '하면서', '비명지른', '초등학생', '기억나네']",
 "['[김승하]', '[오전', '9:05]', '???']",
 "['[김승하]', '[오전', '9:05]', 'ㅋㅋㅋㅋㅋㅋㅋ']",
 "['[김승하]', '[오전', '9:05]', '성신에', '잘하는데', '있어요']",
 "['[김승하]', '[오전', '9:05]', '나랑', '같이', '가요']",
 "['[시끄러운나그네]', '[오전

In [4]:
len(talk)

246702

# EDA

# 자음 모음 분해

초성 인덱스 (음절코드 - 44032)//588 
중성 인덱스 (음절코드 - 44032 - (초성 인덱스*588))/28 
종성 인덱스 (음절코드 - 44032 - (초성 인덱스*588) - (중성 인덱스 * 28))

※ 44032는 '가'의 유니코드, 588는 중성(21)과 종성(28)이 다 돌고 난 후의 개수


In [5]:
!pip install jamo

In [6]:
from jamo import j2h 
from jamo import j2hcj 
from itertools import product

# 초성 리스트. 19개 
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ'] 

# 중성 리스트. 21개 
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ'] 

# 종성 리스트. 28개 
JONGSUNG_LIST = ['_', '_ㄱ', '_ㄲ', '_ㄳ', '_ㄴ', '_ㄵ', '_ㄶ', '_ㄷ', '_ㄹ', '_ㄺ', '_ㄻ', '_ㄼ', '_ㄽ', '_ㄾ', '_ㄿ', '_ㅀ', '_ㅁ', '_ㅂ', '_ㅄ', '_ㅅ', '_ㅆ', '_ㅇ', '_ㅈ', '_ㅊ', '_ㅋ', '_ㅌ', '_ㅍ', '_ㅎ']

# JONGSUNG_LIST = ['_', '_ㄱ', '_ㄱ', '_ㄱ', '_ㄴ', '_ㄴ', '_ㄴ', '_ㄷ', '_ㄹ', '_ㄱ', '_ㅁ', '_ㄹ', '_ㄹ', '_ㄹ', '_ㅂ', '_ㄹ', '_ㅁ', '_ㅂ', '_ㅂ', '_ㄷ', '_ㄷ', '_ㅇ', '_ㄷ', '_ㄷ', '_ㄱ', '_ㄷ', '_ㅂ', '_ㄷ']


In [7]:
def korean_jamo(word):
    r_lst = []
    for w in list(word.strip()):
        if '가' <= w <= '힣':
            ch1 = (ord(w) - ord('가'))//588
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2

            r_lst.extend([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.extend([w])
    return r_lst

In [8]:
!pip install nltk

In [11]:
import os
import sys
import urllib.request
from urllib.parse import quote
import pandas as pd
import json
import re
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from networkx.readwrite.gml import Pattern
import re
import nltk
import numpy as np

#문장출력
words=[]
words2=[]
jamos=[]
for i in range(len(talk)):

    tokenizer = RegexpTokenizer("[ㄱ-ㅎ가-힣]+")
    sentences = tokenizer.tokenize(talk[i])
    words.append(list(sentences)) 
    words2.extend(w2f(sentences))
    jamos.extend(list(sentences))

print('words:',words)


jamos=list(map(korean_jamo,jamos))

print('jamo:',jamos)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


words: 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



jamo: 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# 단어 기준 

In [114]:
from gensim.models import Word2Vec

words_model = Word2Vec(sentences=words, window=10,
                 size = 100, min_count=0, iter=5, workers=4, sg=1)

In [115]:
words_model.wv.vocab

{'시끄러운': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45710>,
 '팀브라더스': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45390>,
 '님과': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45810>,
 '카카오톡': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45990>,
 '대화': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45750>,
 '저장한': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45d10>,
 '날짜': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45fd0>,
 '년': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45090>,
 '월': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45c50>,
 '일': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45590>,
 '월요일': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45f50>,
 '김승하': <gensim.models.keyedvectors.Vocab at 0x7f0c98b454d0>,
 '오전': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45210>,
 '아거': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45290>,
 '귀찌는': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45110>,
 '없지않나요': <gensim.models.keyedvectors.Vocab at 0x7f0c98b45a50>,
 '민영': <gensi

In [126]:
words_model.most_similar(positive=['퇴근'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('출근', 0.9850009679794312),
 ('저녁', 0.9771316051483154),
 ('잤는데', 0.9757493734359741),
 ('담주', 0.9736794233322144),
 ('오늘만', 0.9735746383666992),
 ('어제는', 0.9733207821846008),
 ('왔네요', 0.9725443124771118),
 ('축하해요', 0.9723833799362183),
 ('샀는데', 0.9719257354736328),
 ('하루에', 0.9718469977378845)]

In [117]:
from gensim.models import KeyedVectors

words_model.wv.save_word2vec_format('words_w2v')

### metadata.tsv , tensor.tsv 생성

In [118]:
!python -m gensim.scripts.word2vec2tensor -i words_w2v -o words_w2v

2021-12-22 09:03:27,510 - word2vec2tensor - INFO - running /usr/local/lib/python3.7/dist-packages/gensim/scripts/word2vec2tensor.py -i words_w2v -o words_w2v
2021-12-22 09:03:27,510 - utils_any2vec - INFO - loading projection weights from words_w2v
2021-12-22 09:03:53,840 - utils_any2vec - INFO - loaded (235610, 100) matrix from words_w2v
2021-12-22 09:04:15,622 - word2vec2tensor - INFO - 2D tensor file saved to words_w2v_tensor.tsv
2021-12-22 09:04:15,622 - word2vec2tensor - INFO - Tensor metadata file saved to words_w2v_metadata.tsv
2021-12-22 09:04:15,661 - word2vec2tensor - INFO - finished running word2vec2tensor.py


In [16]:
!ls

KakaoTalk_text2.txt  sample_data	     words_w2v_tensor.tsv
KakaoTalk_text3.txt  words_w2v
kakaotalk_text.txt   words_w2v_metadata.tsv


In [24]:
from google.colab import files

files.download('words_w2v_metadata.tsv')
files.download('words_w2v_tensor.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Embedding Projector: 
https://projector.tensorflow.org/

# 글자 기준

In [18]:
letters = []

for i in words:
    for j in i:
        letters.append(list(j))

letters

[['시', '끄', '러', '운'],
 ['팀', '브', '라', '더', '스'],
 ['님', '과'],
 ['카', '카', '오', '톡'],
 ['대', '화'],
 ['저', '장', '한'],
 ['날', '짜'],
 ['년'],
 ['월'],
 ['일'],
 ['월', '요', '일'],
 ['김', '승', '하'],
 ['오', '전'],
 ['아', '거'],
 ['귀', '찌', '는'],
 ['없', '지', '않', '나', '요'],
 ['민', '영'],
 ['오', '전'],
 ['아'],
 ['민', '영'],
 ['오', '전'],
 ['그', '냥'],
 ['귀', '를'],
 ['하'],
 ['을'],
 ['까'],
 ['김', '승', '하'],
 ['오', '전'],
 ['김', '승', '하'],
 ['오', '전'],
 ['귀', '를'],
 ['뚫', '어', '봐', '요'],
 ['민', '영', '님'],
 ['서', '누', '기'],
 ['인', '나', '청'],
 ['오', '전'],
 ['그', '것', '도', '힘', '들', '져'],
 ['서', '누', '기'],
 ['인', '나', '청'],
 ['오', '전'],
 ['어', '린', '이', '날', '은', '주', '말', '이', '니'],
 ['시', '끄', '러', '운', '나', '그', '네'],
 ['오', '전'],
 ['시', '끄', '러', '운', '나', '그', '네'],
 ['오', '전'],
 ['귀', '뚫', '을', '때'],
 ['민', '영'],
 ['오', '전'],
 ['그', '를', '까'],
 ['민', '영'],
 ['오', '전'],
 ['요'],
 ['시', '끄', '러', '운', '나', '그', '네'],
 ['오', '전'],
 ['뚫'],
 ['김', '승', '하'],
 ['오', '전'],
 ['시', '끄', '러', '운', '나', '그', '네'],
 

In [119]:
from gensim.models import Word2Vec

letters_model = Word2Vec(sentences=letters, window=10,
                 size = 100, min_count=100, iter=5, workers=4, sg=1)

In [120]:
letters_model.most_similar(positive=['삿'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('샀', 0.7321179509162903),
 ('봣', 0.6207128167152405),
 ('탔', 0.5852785110473633),
 ('졋', 0.5764895677566528),
 ('봤', 0.5464346408843994),
 ('됬', 0.5190097093582153),
 ('씩', 0.5174975395202637),
 ('엇', 0.5006145238876343),
 ('떳', 0.49821943044662476),
 ('긁', 0.49620744585990906)]

In [121]:
from gensim.models import KeyedVectors

letters_model.wv.save_word2vec_format('letters_w2v')

#### metadata.tsv , tensor.tsv 생성

In [122]:
!python -m gensim.scripts.word2vec2tensor -i letters_w2v -o letters_w2v

2021-12-22 09:04:44,027 - word2vec2tensor - INFO - running /usr/local/lib/python3.7/dist-packages/gensim/scripts/word2vec2tensor.py -i letters_w2v -o letters_w2v
2021-12-22 09:04:44,027 - utils_any2vec - INFO - loading projection weights from letters_w2v
2021-12-22 09:04:44,135 - utils_any2vec - INFO - loaded (910, 100) matrix from letters_w2v
2021-12-22 09:04:44,217 - word2vec2tensor - INFO - 2D tensor file saved to letters_w2v_tensor.tsv
2021-12-22 09:04:44,217 - word2vec2tensor - INFO - Tensor metadata file saved to letters_w2v_metadata.tsv
2021-12-22 09:04:44,217 - word2vec2tensor - INFO - finished running word2vec2tensor.py


In [30]:
!ls

kakaotalk_text.txt	  letters_w2v_tensor.tsv  words_w2v_metadata.tsv
letters_w2v		  sample_data		  words_w2v_tensor.tsv
letters_w2v_metadata.tsv  words_w2v


#### Embedding Projector:
https://projector.tensorflow.org/

## 단어별

In [123]:
import copy

# 단어별 최소단위로 나누기 
def w2f(text):
    alphabat = []
    vector= []
    alphabat.extend(map(korean_jamo,text))

    return list(np.array(alphabat).flatten())

In [124]:
words2

[['ㅅ', 'ㅣ', '_', 'ㄲ', 'ㅡ', '_', 'ㄹ', 'ㅓ', '_', 'ㅇ', 'ㅜ', '_ㄴ'],
 ['ㅌ', 'ㅣ', '_ㅁ', 'ㅂ', 'ㅡ', '_', 'ㄹ', 'ㅏ', '_', 'ㄷ', 'ㅓ', '_', 'ㅅ', 'ㅡ', '_'],
 ['ㄴ', 'ㅣ', '_ㅁ', 'ㄱ', 'ㅘ', '_'],
 ['ㅋ', 'ㅏ', '_', 'ㅋ', 'ㅏ', '_', 'ㅇ', 'ㅗ', '_', 'ㅌ', 'ㅗ', '_ㄱ'],
 ['ㄷ', 'ㅐ', '_', 'ㅎ', 'ㅘ', '_'],
 ['ㅈ', 'ㅓ', '_', 'ㅈ', 'ㅏ', '_ㅇ', 'ㅎ', 'ㅏ', '_ㄴ'],
 ['ㄴ', 'ㅏ', '_ㄹ', 'ㅉ', 'ㅏ', '_'],
 ['ㄴ', 'ㅕ', '_ㄴ'],
 ['ㅇ', 'ㅝ', '_ㄹ'],
 ['ㅇ', 'ㅣ', '_ㄹ'],
 ['ㅇ', 'ㅝ', '_ㄹ', 'ㅇ', 'ㅛ', '_', 'ㅇ', 'ㅣ', '_ㄹ'],
 ['ㄱ', 'ㅣ', '_ㅁ', 'ㅅ', 'ㅡ', '_ㅇ', 'ㅎ', 'ㅏ', '_'],
 ['ㅇ', 'ㅗ', '_', 'ㅈ', 'ㅓ', '_ㄴ'],
 ['ㅇ', 'ㅏ', '_', 'ㄱ', 'ㅓ', '_'],
 ['ㄱ', 'ㅟ', '_', 'ㅉ', 'ㅣ', '_', 'ㄴ', 'ㅡ', '_ㄴ'],
 ['ㅇ', 'ㅓ', '_ㅄ', 'ㅈ', 'ㅣ', '_', 'ㅇ', 'ㅏ', '_ㄶ', 'ㄴ', 'ㅏ', '_', 'ㅇ', 'ㅛ', '_'],
 ['ㅁ', 'ㅣ', '_ㄴ', 'ㅇ', 'ㅕ', '_ㅇ'],
 ['ㅇ', 'ㅗ', '_', 'ㅈ', 'ㅓ', '_ㄴ'],
 ['ㅇ', 'ㅏ', '_'],
 ['ㅁ', 'ㅣ', '_ㄴ', 'ㅇ', 'ㅕ', '_ㅇ'],
 ['ㅇ', 'ㅗ', '_', 'ㅈ', 'ㅓ', '_ㄴ'],
 ['ㄱ', 'ㅡ', '_', 'ㄴ', 'ㅑ', '_ㅇ'],
 ['ㄱ', 'ㅟ', '_', 'ㄹ', 'ㅡ', '_ㄹ'],
 ['ㅎ', 'ㅏ', '_'],
 ['ㅇ', 'ㅡ', '_ㄹ'],
 ['ㄲ', 'ㅏ', '_'],
 ['ㄱ', 'ㅣ'

In [125]:
from gensim.models import Word2Vec

words2_model = Word2Vec(sentences=words2, window=10,
                 size = 100, min_count=100, iter=5, workers=4, sg=1)

In [26]:
words2_model.wv.vocab

{'_': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd84210>,
 '_ㄱ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9a050>,
 '_ㄲ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9a9d0>,
 '_ㄴ': <gensim.models.keyedvectors.Vocab at 0x7f0d7e6a6750>,
 '_ㄵ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9ab10>,
 '_ㄶ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9a350>,
 '_ㄷ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9a890>,
 '_ㄹ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9a190>,
 '_ㄺ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9aad0>,
 '_ㄻ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9aa90>,
 '_ㄼ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9a950>,
 '_ㅀ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9a450>,
 '_ㅁ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9e5d0>,
 '_ㅂ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9a5d0>,
 '_ㅄ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9a310>,
 '_ㅅ': <gensim.models.keyedvectors.Vocab at 0x7f0d7dd9a4d0>,
 '_ㅆ': <gensim.models.key

In [129]:
from gensim.models import KeyedVectors

words2_model.wv.save_word2vec_format('words2_w2v')

In [130]:
!python -m gensim.scripts.word2vec2tensor -i words2_w2v -o words2_w2v

2021-12-22 09:05:35,977 - word2vec2tensor - INFO - running /usr/local/lib/python3.7/dist-packages/gensim/scripts/word2vec2tensor.py -i words2_w2v -o words2_w2v
2021-12-22 09:05:35,978 - utils_any2vec - INFO - loading projection weights from words2_w2v
2021-12-22 09:05:35,985 - utils_any2vec - INFO - loaded (64, 100) matrix from words2_w2v
2021-12-22 09:05:35,991 - word2vec2tensor - INFO - 2D tensor file saved to words2_w2v_tensor.tsv
2021-12-22 09:05:35,991 - word2vec2tensor - INFO - Tensor metadata file saved to words2_w2v_metadata.tsv
2021-12-22 09:05:35,991 - word2vec2tensor - INFO - finished running word2vec2tensor.py


# 자음,모음 기준

In [31]:
jamos

[['ㅅ', 'ㅣ', '_', 'ㄲ', 'ㅡ', '_', 'ㄹ', 'ㅓ', '_', 'ㅇ', 'ㅜ', '_ㄴ'],
 ['ㅌ', 'ㅣ', '_ㅁ', 'ㅂ', 'ㅡ', '_', 'ㄹ', 'ㅏ', '_', 'ㄷ', 'ㅓ', '_', 'ㅅ', 'ㅡ', '_'],
 ['ㄴ', 'ㅣ', '_ㅁ', 'ㄱ', 'ㅘ', '_'],
 ['ㅋ', 'ㅏ', '_', 'ㅋ', 'ㅏ', '_', 'ㅇ', 'ㅗ', '_', 'ㅌ', 'ㅗ', '_ㄱ'],
 ['ㄷ', 'ㅐ', '_', 'ㅎ', 'ㅘ', '_'],
 ['ㅈ', 'ㅓ', '_', 'ㅈ', 'ㅏ', '_ㅇ', 'ㅎ', 'ㅏ', '_ㄴ'],
 ['ㄴ', 'ㅏ', '_ㄹ', 'ㅉ', 'ㅏ', '_'],
 ['ㄴ', 'ㅕ', '_ㄴ'],
 ['ㅇ', 'ㅝ', '_ㄹ'],
 ['ㅇ', 'ㅣ', '_ㄹ'],
 ['ㅇ', 'ㅝ', '_ㄹ', 'ㅇ', 'ㅛ', '_', 'ㅇ', 'ㅣ', '_ㄹ'],
 ['ㄱ', 'ㅣ', '_ㅁ', 'ㅅ', 'ㅡ', '_ㅇ', 'ㅎ', 'ㅏ', '_'],
 ['ㅇ', 'ㅗ', '_', 'ㅈ', 'ㅓ', '_ㄴ'],
 ['ㅇ', 'ㅏ', '_', 'ㄱ', 'ㅓ', '_'],
 ['ㄱ', 'ㅟ', '_', 'ㅉ', 'ㅣ', '_', 'ㄴ', 'ㅡ', '_ㄴ'],
 ['ㅇ', 'ㅓ', '_ㅄ', 'ㅈ', 'ㅣ', '_', 'ㅇ', 'ㅏ', '_ㄶ', 'ㄴ', 'ㅏ', '_', 'ㅇ', 'ㅛ', '_'],
 ['ㅁ', 'ㅣ', '_ㄴ', 'ㅇ', 'ㅕ', '_ㅇ'],
 ['ㅇ', 'ㅗ', '_', 'ㅈ', 'ㅓ', '_ㄴ'],
 ['ㅇ', 'ㅏ', '_'],
 ['ㅁ', 'ㅣ', '_ㄴ', 'ㅇ', 'ㅕ', '_ㅇ'],
 ['ㅇ', 'ㅗ', '_', 'ㅈ', 'ㅓ', '_ㄴ'],
 ['ㄱ', 'ㅡ', '_', 'ㄴ', 'ㅑ', '_ㅇ'],
 ['ㄱ', 'ㅟ', '_', 'ㄹ', 'ㅡ', '_ㄹ'],
 ['ㅎ', 'ㅏ', '_'],
 ['ㅇ', 'ㅡ', '_ㄹ'],
 ['ㄲ', 'ㅏ', '_'],
 ['ㄱ', 'ㅣ'

In [131]:
from gensim.models import Word2Vec

jamos_model = Word2Vec(sentences=jamos, window=10,
                 size = 100, min_count=0, workers=4, sg=1)

In [132]:
from gensim.models import KeyedVectors

jamos_model.wv.save_word2vec_format('jamos_w2v')

### metadata.tsv , tensor.tsv 생성

In [133]:
!python -m gensim.scripts.word2vec2tensor -i jamos_w2v -o jamos_w2v

2021-12-22 09:06:13,872 - word2vec2tensor - INFO - running /usr/local/lib/python3.7/dist-packages/gensim/scripts/word2vec2tensor.py -i jamos_w2v -o jamos_w2v
2021-12-22 09:06:13,872 - utils_any2vec - INFO - loading projection weights from jamos_w2v
2021-12-22 09:06:13,881 - utils_any2vec - INFO - loaded (73, 100) matrix from jamos_w2v
2021-12-22 09:06:13,888 - word2vec2tensor - INFO - 2D tensor file saved to jamos_w2v_tensor.tsv
2021-12-22 09:06:13,888 - word2vec2tensor - INFO - Tensor metadata file saved to jamos_w2v_metadata.tsv
2021-12-22 09:06:13,888 - word2vec2tensor - INFO - finished running word2vec2tensor.py


In [35]:
!ls

jamos_w2v		letters_w2v		  words_w2v
jamos_w2v_metadata.tsv	letters_w2v_metadata.tsv  words_w2v_metadata.tsv
jamos_w2v_tensor.tsv	letters_w2v_tensor.tsv	  words_w2v_tensor.tsv
kakaotalk_text.txt	sample_data


In [ ]:
from google.colab import files

files.download('jamos_w2v_metadata.tsv')
files.download('jamos_w2v_tensor.tsv')

#### Embedding Projector:
https://projector.tensorflow.org/

# LSTM

#### W2V 데이터를 LSTM 모델의 임베딩 레이어에 넣기 

In [134]:
pretrained_weights = jamos_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape

print('Result embedding shape:', pretrained_weights.shape)
print('Checking similar words:')

Result embedding shape: (73, 100)
Checking similar words:


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [135]:
# Code tried to prepare LSTM model for word generation
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.layers import Dense, Activation

embedding_layer = Embedding(vocab_size, 100, 
                    weights=[pretrained_weights],
                    input_length=1000)

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=vocab_size))   
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

#### word2vec 결과로 데이터 구성

In [48]:
def jamo2idx(word):
  return jamos_model.wv.vocab[word].index

def idx2jamo(idx):
  return jamos_model.wv.index2word[idx]
ex_jamos = jamos[:1000]
train_x = np.zeros([len(ex_jamos), 1000], dtype=np.int32)
train_y = np.zeros([len(ex_jamos) ], dtype=np.int32)
next_spells = []


In [49]:
train_x.shape

(1000, 1000)

In [50]:
train_y.shape

(1000,)

In [54]:
model.fit(train_x, train_y, epochs=1)

32/32 [==============================] - 171s 5s/step - loss: 1.0667


#### 샘플생성

In [89]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_next(inputs, num_generated=100):
    jamos = w2f(inputs)
    print(jamos)
    for i in jamos:
        print(i)
        jamo_idxs = jamo2idx(i)
        print(jamo_idxs)
        prediction = model.predict(x=np.array(jamo_idxs))
        print(prediction)
        idx = sample(prediction[-1], temperature=0.7)
        print(idx)
        jamo_idxs.append(idx)
    return ' '.join(idx2jamo(idx) for idx in jamo_idxs)

In [91]:
generate_next('하')

['ㅎ', 'ㅏ', '_']
ㅎ
8


IndexError: ignored

# INPUT

In [108]:
# 확인할 문장 
def input_texts(texts):

    texts = texts.split()
    
    input_words=[]
    input_words2=[]
    input_letters=[]
    input_jamos=[]
    
    for i in range(len(texts)):
        tokenizer = RegexpTokenizer("[ㄱ-ㅎ가-힣]+")
        text = tokenizer.tokenize(texts[i])
        input_words.append(list(text)) 
        input_words2.extend(w2f(text))
        input_jamos.extend(list(text))

    for i in input_words:
        for j in i:
            input_letters.append(list(j))

    input_jamos=list(map(korean_jamo,input_jamos))

    return input_words, input_words2, input_letters, input_jamos


In [109]:
inputs='오늘은 배가 너무 고파서 힘들어'
input_words, input_words2, input_letters, input_jamos = input_texts(inputs)

In [110]:
print('input_words:',input_words)
print('input_words2:',input_words2)
print('input_letters:',input_letters)
print('input_jamos:',input_jamos)

input_words: [['오늘은'], ['배가'], ['너무'], ['고파서'], ['힘들어']]
input_words2: ['ㅇ', 'ㅗ', '_', 'ㄴ', 'ㅡ', '_ㄹ', 'ㅇ', 'ㅡ', '_ㄴ', 'ㅂ', 'ㅐ', '_', 'ㄱ', 'ㅏ', '_', 'ㄴ', 'ㅓ', '_', 'ㅁ', 'ㅜ', '_', 'ㄱ', 'ㅗ', '_', 'ㅍ', 'ㅏ', '_', 'ㅅ', 'ㅓ', '_', 'ㅎ', 'ㅣ', '_ㅁ', 'ㄷ', 'ㅡ', '_ㄹ', 'ㅇ', 'ㅓ', '_']
input_letters: [['오', '늘', '은'], ['배', '가'], ['너', '무'], ['고', '파', '서'], ['힘', '들', '어']]
input_jamos: [['ㅇ', 'ㅗ', '_', 'ㄴ', 'ㅡ', '_ㄹ', 'ㅇ', 'ㅡ', '_ㄴ'], ['ㅂ', 'ㅐ', '_', 'ㄱ', 'ㅏ', '_'], ['ㄴ', 'ㅓ', '_', 'ㅁ', 'ㅜ', '_'], ['ㄱ', 'ㅗ', '_', 'ㅍ', 'ㅏ', '_', 'ㅅ', 'ㅓ', '_'], ['ㅎ', 'ㅣ', '_ㅁ', 'ㄷ', 'ㅡ', '_ㄹ', 'ㅇ', 'ㅓ', '_']]


In [112]:
words_model.most_similar(positive = ['오늘은','맛이','너무','없어서','힘들어'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('나가있었어용', 0.7214723825454712),
 ('밍밍해서', 0.7088960409164429),
 ('아닌맛이라', 0.7038617134094238),
 ('장변동이', 0.7005026340484619),
 ('안오더라규요', 0.6973728537559509),
 ('당한기억밖에', 0.6897611618041992),
 ('부신', 0.6893656253814697),
 ('빨리가요', 0.6881861686706543),
 ('줄여야겠다고', 0.6870722770690918),
 ('가볼라거여', 0.6868160963058472)]

In [98]:
words2_model.most_similar(positive= w2f('오늘'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('ㅏ', 0.6186736822128296),
 ('ㄷ', 0.5568584203720093),
 ('ㄱ', 0.5522733926773071),
 ('ㅅ', 0.5367653965950012),
 ('_ㅅ', 0.5123903751373291),
 ('_ㄴ', 0.5111385583877563),
 ('ㅣ', 0.5078257322311401),
 ('ㅔ', 0.5028915405273438),
 ('ㅓ', 0.49855661392211914),
 ('ㄹ', 0.4872031807899475)]

In [92]:
letters_model.most_similar(positive = ['좋','아','용'], negative= ['좋','아'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('요', 0.5546654462814331),
 ('욤', 0.538600742816925),
 ('욬', 0.5072736740112305),
 ('ㅎ', 0.38080304861068726),
 ('여', 0.3778068721294403),
 ('옄', 0.36881178617477417),
 ('ㅂ', 0.35988155007362366),
 ('흔', 0.33087337017059326),
 ('혈', 0.32691580057144165),
 ('염', 0.3140566349029541)]